# マルチモーダルな世界モデル

マルチモーダルな世界モデルでは、未来予測を使った意思決定を、実装可能な形に分解して学びます。

前提: 時系列モデリングと強化学習の基礎があると効果的です。

到達目標: 世界モデルを、状態表現・遷移・観測予測・計画の連鎖として理解し、実験設計へ落とし込める状態にします。

ここで扱う中心語は 「状態表現」、「遷移モデル」、「観測予測」、「計画」、「ロールアウト」、「マルチモーダルな世界モデル」 です。用語を先に暗記するのではなく、コード実行の結果と結びつけて理解します。

このノートは 世界モデル 分野の初学者向けに、説明とコードを交互に読み進める設計です。最初から完璧に理解する必要はありません。大切なのは、各コードの目的を一文で言えることと、出力が変わる理由を自分で確かめることです。

## 1. 状態遷移の最小モデル

世界モデルの最初の核は遷移式です。状態と行動から次状態を作る操作を最小形で確認します。

ここで注目してほしいのは、処理の入出力がどの変数で接続されるかです。コードを読むときは『この行は何を決める行か』を短くメモしながら進むと理解が安定します。

In [ ]:
z_t = 0.25
a_t = 1.0
A, B = 0.92, 0.18
z_next = A * z_t + B * a_t
print('z_next =', round(z_next, 6))

遷移式の係数は環境のダイナミクス仮定を表します。係数の選び方で予測性質が大きく変わります。

たとえば 状態表現 を別の値に変更すると、出力がどう変わるかを確認してください。次に、同じコードで境界条件を試し、なぜその結果になるのかを言葉で説明します。

この確認を繰り返すと、理論と実装の対応が頭の中で一本化されます。

## 2. 観測予測を作る

次に、潜在状態から観測を復元する写像を作ります。状態推定と観測再現の役割分担をコードで掴みます。

ここで注目してほしいのは、処理の入出力がどの変数で接続されるかです。コードを読むときは『この行は何を決める行か』を短くメモしながら進むと理解が安定します。

In [ ]:
def decode(z):
    return {'position': 2.5 * z + 0.1, 'velocity': 0.8 * z - 0.05}
obs_next = decode(z_next)
print('obs_next =', {k: round(v, 4) for k, v in obs_next.items()})
print('keys =', list(obs_next.keys()))

観測予測を別関数に切ると、遷移誤差と観測誤差を分離して調整できます。

たとえば 状態表現 を別の値に変更すると、出力がどう変わるかを確認してください。次に、同じコードで境界条件を試し、なぜその結果になるのかを言葉で説明します。

この確認を繰り返すと、理論と実装の対応が頭の中で一本化されます。

## 式をコードと接続する

ここで、本文中で使っている式を実装視点で再整理します。式を暗記するのではなく、どの変数がどのコード行に対応するかを確認してください。

1. z_{t+1} = f_theta(z_t, a_t)
2. o_hat_{t+1} = g_theta(z_{t+1})

この一覧は『正解の丸暗記』ではなく、デバッグ時の観点表です。実装が壊れたとき、どの式のどの項が怪しいかを逆算できるように使います。

## 3. ロールアウトを試す

ここで複数ステップ予測を実行します。1ステップでは見えない誤差累積を把握するためです。

ここで注目してほしいのは、処理の入出力がどの変数で接続されるかです。コードを読むときは『この行は何を決める行か』を短くメモしながら進むと理解が安定します。

In [ ]:
actions = [0.0, 1.0, 1.0, 0.0, -0.5]
z = 0.1
traj = []
for a in actions:
    z = 0.92 * z + 0.18 * a
    traj.append(round(z, 5))
print('rollout =', traj)

長期予測で崩れるなら、遷移モデルの安定性や状態表現の情報量不足を疑います。

たとえば 状態表現 を別の値に変更すると、出力がどう変わるかを確認してください。次に、同じコードで境界条件を試し、なぜその結果になるのかを言葉で説明します。

この確認を繰り返すと、理論と実装の対応が頭の中で一本化されます。

## 4. 計画候補を比較する

次に、複数の行動列を比較して、どの計画が望ましいかを評価します。モデルベース強化学習の中心操作です。

ここで注目してほしいのは、処理の入出力がどの変数で接続されるかです。コードを読むときは『この行は何を決める行か』を短くメモしながら進むと理解が安定します。

In [ ]:
plans = [[0, 1, 1], [1, 1, 1], [0, 0, 1]]
def score_plan(plan):
    z = 0.1
    for a in plan:
        z = 0.92 * z + 0.18 * a
    return z
scores = [round(score_plan(p), 5) for p in plans]
print('scores =', scores)

計画評価が可能になると、実環境での試行回数を抑えた探索がしやすくなります。

たとえば 状態表現 を別の値に変更すると、出力がどう変わるかを確認してください。次に、同じコードで境界条件を試し、なぜその結果になるのかを言葉で説明します。

この確認を繰り返すと、理論と実装の対応が頭の中で一本化されます。

## 5. モデル誤差を監視する

最後に、予測と実測の差を定量化します。世界モデルは『予測できる範囲』を常に点検する運用が重要です。

ここで注目してほしいのは、処理の入出力がどの変数で接続されるかです。コードを読むときは『この行は何を決める行か』を短くメモしながら進むと理解が安定します。

In [ ]:
pred = [0.10, 0.22, 0.31, 0.29]
real = [0.11, 0.25, 0.28, 0.35]
errors = [abs(p - r) for p, r in zip(pred, real)]
print('errors =', [round(e, 4) for e in errors])
print('mean_error =', round(sum(errors) / len(errors), 5))

平均誤差だけでなく時点別誤差を追うと、どの遷移条件でモデルが弱いかを特定しやすくなります。

たとえば 状態表現 を別の値に変更すると、出力がどう変わるかを確認してください。次に、同じコードで境界条件を試し、なぜその結果になるのかを言葉で説明します。

この確認を繰り返すと、理論と実装の対応が頭の中で一本化されます。

## つまずきポイントと確認課題

実装学習では、書けることより説明できることを重視します。次の失敗パターンを先に知っておくと、学習速度が上がります。

1. 1ステップ誤差だけで安心してしまう
2. 長期予測の誤差爆発を監視しない
3. 状態表現が不足しているのにモデルだけ調整する

思考実験: 同じ行動列で、状態表現の次元だけ変えたときの予測劣化を想像します。

確認課題
1. このノートのコードのうち1つを選び、行ごとの役割を口頭で説明する。
2. 定数を1つ変更し、出力変化の理由を『入力 -> 中間値 -> 出力』で説明する。
3. 実務利用を想定し、異常入力時に壊れる箇所と防止策を1つ提案する。

マルチモーダルな世界モデル の学習はここで一区切りです。前のノートに戻り、同じ観点でコードを読み直すと体系として定着します。

補講 1

世界モデル の学習で最も効果があるのは、コードを読んだあとに式へ戻る往復です。先に式だけを追うと抽象度が高く、初学者は何を計算しているかを見失いやすくなります。逆に、先にコードを動かして値の流れを掴んでから式へ戻ると、記号の役割が具体的に見えてきます。

補講 2

デバッグ時の基本姿勢は、入力・中間値・出力の三点を固定して比較することです。うまく動かないとき、いきなり大きな修正を入れると原因を見失います。まずは1つの定数だけを変更し、どの中間値が変わったかを観察してください。この小さな観察を積み重ねることが、実装力を最短で伸ばす道です。

補講 3

初学者がつまずく典型は、正しいコードと正しい説明を別々に持ってしまうことです。コードは動くが言葉で説明できない、あるいは説明はできるがコードに落とせない、という分離が起きます。これを防ぐには、実行後に『この行は何を更新したか』を一文で言う練習が有効です。

補講 4

評価設計の観点でも、実装の意味を言葉で固定することが重要です。単一の数値だけを見ると、改善の方向を誤ることがあります。できるだけ複数の観点、例えば平均誤差と外れ値、短期挙動と長期挙動を同時に観察し、どの観点で改善したかを明示してください。

補講 5

世界モデル の学習で最も効果があるのは、コードを読んだあとに式へ戻る往復です。先に式だけを追うと抽象度が高く、初学者は何を計算しているかを見失いやすくなります。逆に、先にコードを動かして値の流れを掴んでから式へ戻ると、記号の役割が具体的に見えてきます。

補講 6

デバッグ時の基本姿勢は、入力・中間値・出力の三点を固定して比較することです。うまく動かないとき、いきなり大きな修正を入れると原因を見失います。まずは1つの定数だけを変更し、どの中間値が変わったかを観察してください。この小さな観察を積み重ねることが、実装力を最短で伸ばす道です。

補講 7

初学者がつまずく典型は、正しいコードと正しい説明を別々に持ってしまうことです。コードは動くが言葉で説明できない、あるいは説明はできるがコードに落とせない、という分離が起きます。これを防ぐには、実行後に『この行は何を更新したか』を一文で言う練習が有効です。

補講 8

評価設計の観点でも、実装の意味を言葉で固定することが重要です。単一の数値だけを見ると、改善の方向を誤ることがあります。できるだけ複数の観点、例えば平均誤差と外れ値、短期挙動と長期挙動を同時に観察し、どの観点で改善したかを明示してください。

補講 9

世界モデル の学習で最も効果があるのは、コードを読んだあとに式へ戻る往復です。先に式だけを追うと抽象度が高く、初学者は何を計算しているかを見失いやすくなります。逆に、先にコードを動かして値の流れを掴んでから式へ戻ると、記号の役割が具体的に見えてきます。

補講 10

デバッグ時の基本姿勢は、入力・中間値・出力の三点を固定して比較することです。うまく動かないとき、いきなり大きな修正を入れると原因を見失います。まずは1つの定数だけを変更し、どの中間値が変わったかを観察してください。この小さな観察を積み重ねることが、実装力を最短で伸ばす道です。

補講 11

初学者がつまずく典型は、正しいコードと正しい説明を別々に持ってしまうことです。コードは動くが言葉で説明できない、あるいは説明はできるがコードに落とせない、という分離が起きます。これを防ぐには、実行後に『この行は何を更新したか』を一文で言う練習が有効です。

補講 12

評価設計の観点でも、実装の意味を言葉で固定することが重要です。単一の数値だけを見ると、改善の方向を誤ることがあります。できるだけ複数の観点、例えば平均誤差と外れ値、短期挙動と長期挙動を同時に観察し、どの観点で改善したかを明示してください。

補講 13

世界モデル の学習で最も効果があるのは、コードを読んだあとに式へ戻る往復です。先に式だけを追うと抽象度が高く、初学者は何を計算しているかを見失いやすくなります。逆に、先にコードを動かして値の流れを掴んでから式へ戻ると、記号の役割が具体的に見えてきます。

補講 14

デバッグ時の基本姿勢は、入力・中間値・出力の三点を固定して比較することです。うまく動かないとき、いきなり大きな修正を入れると原因を見失います。まずは1つの定数だけを変更し、どの中間値が変わったかを観察してください。この小さな観察を積み重ねることが、実装力を最短で伸ばす道です。

補講 15

初学者がつまずく典型は、正しいコードと正しい説明を別々に持ってしまうことです。コードは動くが言葉で説明できない、あるいは説明はできるがコードに落とせない、という分離が起きます。これを防ぐには、実行後に『この行は何を更新したか』を一文で言う練習が有効です。

補講 16

評価設計の観点でも、実装の意味を言葉で固定することが重要です。単一の数値だけを見ると、改善の方向を誤ることがあります。できるだけ複数の観点、例えば平均誤差と外れ値、短期挙動と長期挙動を同時に観察し、どの観点で改善したかを明示してください。

補講 1

世界モデル の学習で最も効果があるのは、コードを読んだあとに式へ戻る往復です。先に式だけを追うと抽象度が高く、初学者は何を計算しているかを見失いやすくなります。逆に、先にコードを動かして値の流れを掴んでから式へ戻ると、記号の役割が具体的に見えてきます。

補講 2

デバッグ時の基本姿勢は、入力・中間値・出力の三点を固定して比較することです。うまく動かないとき、いきなり大きな修正を入れると原因を見失います。まずは1つの定数だけを変更し、どの中間値が変わったかを観察してください。この小さな観察を積み重ねることが、実装力を最短で伸ばす道です。

補講 3

初学者がつまずく典型は、正しいコードと正しい説明を別々に持ってしまうことです。コードは動くが言葉で説明できない、あるいは説明はできるがコードに落とせない、という分離が起きます。これを防ぐには、実行後に『この行は何を更新したか』を一文で言う練習が有効です。

補講 4

評価設計の観点でも、実装の意味を言葉で固定することが重要です。単一の数値だけを見ると、改善の方向を誤ることがあります。できるだけ複数の観点、例えば平均誤差と外れ値、短期挙動と長期挙動を同時に観察し、どの観点で改善したかを明示してください。

補講 5

世界モデル の学習で最も効果があるのは、コードを読んだあとに式へ戻る往復です。先に式だけを追うと抽象度が高く、初学者は何を計算しているかを見失いやすくなります。逆に、先にコードを動かして値の流れを掴んでから式へ戻ると、記号の役割が具体的に見えてきます。

補講 6

デバッグ時の基本姿勢は、入力・中間値・出力の三点を固定して比較することです。うまく動かないとき、いきなり大きな修正を入れると原因を見失います。まずは1つの定数だけを変更し、どの中間値が変わったかを観察してください。この小さな観察を積み重ねることが、実装力を最短で伸ばす道です。

補講 7

初学者がつまずく典型は、正しいコードと正しい説明を別々に持ってしまうことです。コードは動くが言葉で説明できない、あるいは説明はできるがコードに落とせない、という分離が起きます。これを防ぐには、実行後に『この行は何を更新したか』を一文で言う練習が有効です。

補講 8

評価設計の観点でも、実装の意味を言葉で固定することが重要です。単一の数値だけを見ると、改善の方向を誤ることがあります。できるだけ複数の観点、例えば平均誤差と外れ値、短期挙動と長期挙動を同時に観察し、どの観点で改善したかを明示してください。

補講 9

世界モデル の学習で最も効果があるのは、コードを読んだあとに式へ戻る往復です。先に式だけを追うと抽象度が高く、初学者は何を計算しているかを見失いやすくなります。逆に、先にコードを動かして値の流れを掴んでから式へ戻ると、記号の役割が具体的に見えてきます。

補講 10

デバッグ時の基本姿勢は、入力・中間値・出力の三点を固定して比較することです。うまく動かないとき、いきなり大きな修正を入れると原因を見失います。まずは1つの定数だけを変更し、どの中間値が変わったかを観察してください。この小さな観察を積み重ねることが、実装力を最短で伸ばす道です。

補講 11

初学者がつまずく典型は、正しいコードと正しい説明を別々に持ってしまうことです。コードは動くが言葉で説明できない、あるいは説明はできるがコードに落とせない、という分離が起きます。これを防ぐには、実行後に『この行は何を更新したか』を一文で言う練習が有効です。

補講 12

評価設計の観点でも、実装の意味を言葉で固定することが重要です。単一の数値だけを見ると、改善の方向を誤ることがあります。できるだけ複数の観点、例えば平均誤差と外れ値、短期挙動と長期挙動を同時に観察し、どの観点で改善したかを明示してください。

補講 13

世界モデル の学習で最も効果があるのは、コードを読んだあとに式へ戻る往復です。先に式だけを追うと抽象度が高く、初学者は何を計算しているかを見失いやすくなります。逆に、先にコードを動かして値の流れを掴んでから式へ戻ると、記号の役割が具体的に見えてきます。

補講 14

デバッグ時の基本姿勢は、入力・中間値・出力の三点を固定して比較することです。うまく動かないとき、いきなり大きな修正を入れると原因を見失います。まずは1つの定数だけを変更し、どの中間値が変わったかを観察してください。この小さな観察を積み重ねることが、実装力を最短で伸ばす道です。

補講 15

初学者がつまずく典型は、正しいコードと正しい説明を別々に持ってしまうことです。コードは動くが言葉で説明できない、あるいは説明はできるがコードに落とせない、という分離が起きます。これを防ぐには、実行後に『この行は何を更新したか』を一文で言う練習が有効です。

補講 16

評価設計の観点でも、実装の意味を言葉で固定することが重要です。単一の数値だけを見ると、改善の方向を誤ることがあります。できるだけ複数の観点、例えば平均誤差と外れ値、短期挙動と長期挙動を同時に観察し、どの観点で改善したかを明示してください。

補講 1

世界モデル の学習で最も効果があるのは、コードを読んだあとに式へ戻る往復です。先に式だけを追うと抽象度が高く、初学者は何を計算しているかを見失いやすくなります。逆に、先にコードを動かして値の流れを掴んでから式へ戻ると、記号の役割が具体的に見えてきます。

補講 2

デバッグ時の基本姿勢は、入力・中間値・出力の三点を固定して比較することです。うまく動かないとき、いきなり大きな修正を入れると原因を見失います。まずは1つの定数だけを変更し、どの中間値が変わったかを観察してください。この小さな観察を積み重ねることが、実装力を最短で伸ばす道です。

補講 3

初学者がつまずく典型は、正しいコードと正しい説明を別々に持ってしまうことです。コードは動くが言葉で説明できない、あるいは説明はできるがコードに落とせない、という分離が起きます。これを防ぐには、実行後に『この行は何を更新したか』を一文で言う練習が有効です。

補講 4

評価設計の観点でも、実装の意味を言葉で固定することが重要です。単一の数値だけを見ると、改善の方向を誤ることがあります。できるだけ複数の観点、例えば平均誤差と外れ値、短期挙動と長期挙動を同時に観察し、どの観点で改善したかを明示してください。

補講 5

世界モデル の学習で最も効果があるのは、コードを読んだあとに式へ戻る往復です。先に式だけを追うと抽象度が高く、初学者は何を計算しているかを見失いやすくなります。逆に、先にコードを動かして値の流れを掴んでから式へ戻ると、記号の役割が具体的に見えてきます。

補講 6

デバッグ時の基本姿勢は、入力・中間値・出力の三点を固定して比較することです。うまく動かないとき、いきなり大きな修正を入れると原因を見失います。まずは1つの定数だけを変更し、どの中間値が変わったかを観察してください。この小さな観察を積み重ねることが、実装力を最短で伸ばす道です。

補講 7

初学者がつまずく典型は、正しいコードと正しい説明を別々に持ってしまうことです。コードは動くが言葉で説明できない、あるいは説明はできるがコードに落とせない、という分離が起きます。これを防ぐには、実行後に『この行は何を更新したか』を一文で言う練習が有効です。

補講 8

評価設計の観点でも、実装の意味を言葉で固定することが重要です。単一の数値だけを見ると、改善の方向を誤ることがあります。できるだけ複数の観点、例えば平均誤差と外れ値、短期挙動と長期挙動を同時に観察し、どの観点で改善したかを明示してください。

補講 9

世界モデル の学習で最も効果があるのは、コードを読んだあとに式へ戻る往復です。先に式だけを追うと抽象度が高く、初学者は何を計算しているかを見失いやすくなります。逆に、先にコードを動かして値の流れを掴んでから式へ戻ると、記号の役割が具体的に見えてきます。